## Installation of Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#installing the huggingsound 
!pip install huggingsound


In [3]:
# import libraries and transformer
import librosa
import torch
from transformers import Wav2Vec2ForCTC,Wav2Vec2Processor #importing algorithms which train the NNs for problems like ASR
import IPython.display as show

##Model Loading

In [9]:
#load the pretrained model and audio file
model_id="BakhtUllah123/xlsr_ur_training"
audio_path="/content/drive/MyDrive/clips/common_voice_ur_26562732.mp3" 


In [5]:
#Display the audio
show.Audio(audio_path,rate=16_000)

In [6]:
!git config --global credential.helper store

In [7]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [10]:
# using the pretrained model in Wav2vecprocessor as processor and Wav2vecforCTC as a model
processor=Wav2Vec2Processor.from_pretrained(model_id,use_auth_token=True)
model=Wav2Vec2ForCTC.from_pretrained(model_id,use_auth_token=True)

## Setup for Evaluation

In [12]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/train.tsv',sep="\t")
test = pd.read_csv('/content/drive/MyDrive/test.tsv',sep="\t")

In [13]:
train_path = train['path'].to_list()
test_path = test['path'].to_list()
all_path = train_path+test_path
len(all_path)

810

In [14]:
train_path = train['sentence'].to_list()
train_path

['پر اپنی محنت سے حاصل کیا ہے',
 'جبکہ مصباح الحق اور یونس خان جیسے سینئرز صرف اس لیے اب تک کھیل رہے ہیں',
 'پنجاب میں صنعتوں اور سی این جی سیکٹرز کو گیس کی فراہمی معطل',
 'لندن فلیٹس کی ملکیت ثابت شدہ ہے۔',
 'کئی کئی برسوں کا فرق ہے۔',
 'تو پاکستان کی سیاسی جماعتیں یہ کیوں نہ کر سکیں؟',
 'جب جنرل راحیل شریف آرمی چیف تھے۔',
 'مگر حیرت کی بات کہ قرآن شریف کو سب سے اوپر رکھا گیا ، گویا سب سےمقدس صحیفہ یہی ہے',
 'گاؤں جہاں کا قصد تھا ، کا نام "کلاں" ہے ۔',
 'اب بھی وہی قائم ہیں۔',
 'ضیاء الحق دور میں قوم بٹی ہوئی تھی۔',
 'اداکار شاہد کپور کے بارے میں بھی',
 'یہ کسی قوم کی ایک اجتماعی ضرورت کا فطری جواب ہے۔',
 'مجھے دفتر پیدل جانا پڑتا ہے',
 'خدانہ کرے کہ ایسا ہو۔',
 'اپنی چالیسویں سالگرہ کی طرف بڑھتے ہوئے ک',
 'بچن نے اپنی بھابھی ایشوریا کے بارے میں',
 'اب جو اعتراضات اٹھائے گئے ہیں۔',
 'میں فلم کو سمجھ نہیں پایا تھا',
 'اوراب وہ رد ہوچکی۔',
 'اچھی بھی نہیں لگتی۔',
 'لیکن ان کی حکومت شخصی حکومت نہ تھی۔',
 'جس کو صائم الدھر کہا جاتا ہے',
 'پرندے نقل مکانی کرنا شروع کرتے ہیں',
 'اقدار میں ا

In [15]:
test_path = test['sentence'].to_list()
test_path

['اور یہ رنگین لمحات آنکھوں اور دل میں محفوظ کر لیا جائیں۔',
 'نئے نوٹوں کی بکنگ اپنی انتہائی حد تک پہنچنے کے بعد بند کر دی جائے گی اسٹیٹ بینک',
 'مانتے ہیں کہ ہر اچھائی خدا کی طرف سے ہے',
 'وحشت اب بوریت سے ہے۔',
 'تو ان کے خلاف مقدمات کا فیصلہ کیوں نہیں ہو رہا؟',
 'اور بعض مسلمان سمجھتے ہیں',
 'وہ بے وفا ثابت ہوا',
 'پاکستان تو اس وقت غیر معمولی حالات سے گزر رہا ہے۔',
 'کانگریس کی مسلم کش پالیسیوں سے بددل ہوکر',
 'پاکستان نہ بنتا۔',
 'ملکی معیشت کی بدلتی صورتحال اور استحکام کی وجوہات کیا ہیں',
 'برصغیرکے مسلمانوں میں بھی ایسے لوگ پائے جاتے ہیں۔',
 'جب جمہوریت کی بات ہوتی ہے۔',
 'نجانے کیوں مجھے ان بچوں میں اپنا بچپن نظر آ رہا تھا',
 'ہم کن اندھیروں میں پھنسے ہوئے ہیں۔',
 'پشاورشہید بے نظیر بھٹو یونیورسٹی میں ووشو چیمپئن شپ کا انعقاد',
 'سروج خان بالی وڈ',
 'چیمپئنز ٹرافی ہاکی گرین شرٹس اپنا پانچواں میچ بیلجیئم کیخلاف کھیلیں گے',
 'یہ صرف نمونہ اور نمائش رہ گئی ہے۔',
 'مو لا نا امین احسن اصلاحی نے جو کام تفسیر میں',
 'تمہاری خوشی میں میری خوشی ہے',
 'اور پھر ایک بہادر ہیرو کے ذریعہ آد

In [16]:
# to extract all the audios inside the test_Audios folder and put them into named audios_path list
import os
audios_path=[]
dir = "/content/drive/MyDrive/clips"
for path in all_path:
  full_path = os.path.join(dir, path)
  if os.path.isfile(full_path):
      audios_path.append(full_path)
len(audios_path)

810

In [17]:
# this will take 10 random audios from the audios_path
import random
my_array=[]
for i in range(10):
  seq=random.choices(audios_path,weights=map(len,audios_path))
  my_array.append(seq)


In [18]:
# to convert nestedlist into a single list
my_arr=[]
for list in my_array:
  my_arr +=list

## Evaluation

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
# to process each item in my_arr with sr=16K and store it into speech_array
# each item will be loaded through librosa library
pred_sent = []
for i in audios_path:
    speech_array,sampling_rate=librosa.load(i,sr=16_000)

# passing speech array into the processor to sample it at a rate of 16K and padding to equal the length
    input_ids = processor(
      speech_array,
      sampling_rate=16_000,
      return_tensors="pt",   # return in pytorch format.
      padding=True
    )

    with torch.no_grad():  # it means that to not calculate the gradient.
      logits=model(              #logits calculate the probability of the input_ids
        input_ids.input_values,
        attention_mask=input_ids.attention_mask  # it tells us the existance of words, 1 show that word is present and vice versa 
    ).logits

    predicted_ids = torch.argmax(logits, dim=-1)    # Returns the indexes of the maximum value of all elements in the input
    predicted_sentences = processor.batch_decode(predicted_ids)  # decode the predicted_ids into sentences
    pred_sent.append(predicted_sentences)


In [ ]:
# to show the pred_sent with large font size
show.Markdown(f'###{pred_sent}')

##Setup for Word Error Rate

In [22]:
#create and open a txt file
file=open("predicted.txt","x") # we pass x for the reason if the exists then it will through an error

In [23]:
# to arrange the sentences into a single list from nested list
empty_list=[]
for i in pred_sent:
  x=(''.join(i))
  empty_list.append(x)


In [ ]:
#to get the sentences line by line without a list
print(*empty_list,sep="\n")

In [25]:
# writing data in predicted.txt file
for index in empty_list:
  file.write(index + '\n')
file.close()

# Calculating Word Error Rate

In [27]:
#importing path, WER and CER from pathlib and jiwer library
from jiwer import wer, cer
from pathlib import Path

# defining path of the txt files
Refrence_path = Path('/content/drive/MyDrive/transcriptions.txt')
prediction_path = Path('/content/predicted.txt')

# converting the file into a python list
Refrence = Refrence_path.read_text().splitlines()  # read the text and spliting a strings into a list of Refrence sentences
prediction = prediction_path.read_text().splitlines()   # read the text and spliting a strings into a list of prediction sentences

print(Refrence)
print(prediction)

word_Error_Rate = wer(Refrence, prediction) # computing WER by comparing refrence and prediction sentences
Character_Error_Rate = cer(Refrence, prediction)   # computing CER by comparing refrence and prediction sentences

print("Word error rate is = {}".format(word_Error_Rate))
print("Character error rate is = {}".format(Character_Error_Rate))

["['پر اپنی محنت سے حاصل کیا ہے',", " 'جبکہ مصباح الحق اور یونس خان جیسے سینئرز صرف اس لیے اب تک کھیل رہے ہیں',", " 'پنجاب میں صنعتوں اور سی این جی سیکٹرز کو گیس کی فراہمی معطل',", " 'لندن فلیٹس کی ملکیت ثابت شدہ ہے۔',", " 'کئی کئی برسوں کا فرق ہے۔',", " 'تو پاکستان کی سیاسی جماعتیں یہ کیوں نہ کر سکیں؟',", " 'جب جنرل راحیل شریف آرمی چیف تھے۔',", " 'مگر حیرت کی بات کہ قرآن شریف کو سب سے اوپر رکھا گیا ، گویا سب سےمقدس صحیفہ یہی ہے',", ' \'گاؤں جہاں کا قصد تھا ، کا نام "کلاں" ہے ۔\',', " 'اب بھی وہی قائم ہیں۔',", " 'ضیاء الحق دور میں قوم بٹی ہوئی تھی۔',", " 'اداکار شاہد کپور کے بارے میں بھی',", " 'یہ کسی قوم کی ایک اجتماعی ضرورت کا فطری جواب ہے۔',", " 'مجھے دفتر پیدل جانا پڑتا ہے',", " 'خدانہ کرے کہ ایسا ہو۔',", " 'اپنی چالیسویں سالگرہ کی طرف بڑھتے ہوئے ک',", " 'بچن نے اپنی بھابھی ایشوریا کے بارے میں',", " 'اب جو اعتراضات اٹھائے گئے ہیں۔',", " 'میں فلم کو سمجھ نہیں پایا تھا',", " 'اوراب وہ رد ہوچکی۔',", " 'اچھی بھی نہیں لگتی۔',", " 'لیکن ان کی حکومت شخصی حکومت نہ تھی۔',", " 'جس کو صائم ال